In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import selenium.webdriver.support.ui as UI
import pandas as pd
import numpy as np
import re
import time




In [26]:
region_array = []
city_array = []

driver = webdriver.Chrome()
driver.get('https://www.ssp.sp.gov.br/Estatistica/Pesquisa.aspx')

month_data = driver.find_element_by_xpath('/html/body/div[3]/div/div[1]/form/div[3]/div[1]/div[5]/div[1]/div/a')
month_data.click()


#-------------------------------------Select de municipios-------------------------------

city_dict = {"Municipio":[],"Valor":[]}

select_city = UI.Select(driver.find_element_by_xpath('//*[@id="conteudo_ddlMunicipios"]'))

for option in select_city.options:
    city_dict['Municipio'].append(option.text)
    city_dict['Valor'].append(int(option.get_attribute('value'))) 

city_dict['Municipio'].pop(0)
city_dict['Valor'].pop(0)



#-------------------------------------Select de municipios-------------------------------

for i,j in zip(city_dict['Valor'],city_dict['Municipio']):

    ## option[3] is 2021, i dont wanna get data from other years for now, so i just keep it
    year = driver.find_element_by_xpath('/html/body/div[3]/div/div[1]/form/div[3]/div[1]/div[2]/div[1]/div/select/option[3]')
    year.click()
    
    time.sleep(2)

    city = driver.find_element_by_xpath('/html/body/div[3]/div/div[1]/form/div[3]/div[1]/div[3]/div[1]/div/select/option[{i}]'.format(i=i+1))
    city.click()

    time.sleep(2)

    select_region = UI.Select(driver.find_element_by_xpath('//*[@id="conteudo_ddlRegioes"]'))
    region = select_region.first_selected_option.text

    down = driver.find_element_by_xpath('/html/body/div[3]/div/div[1]/form/div[3]/div[1]/div[7]/div/div/input')
    down.click()

    time.sleep(2)

    print('data from {j} was sucessfully downloaded -- Region: {region}'.format(j = j,region=region))

    driver.get('https://www.ssp.sp.gov.br/Estatistica/Pesquisa.aspx')

    time.sleep(2)

    month_data = driver.find_element_by_xpath('/html/body/div[3]/div/div[1]/form/div[3]/div[1]/div[5]/div[1]/div/a')
    month_data.click()

    region_array.append(region)
    city_array.append(j)




driver.close()

data from Adamantina was sucessfully downloaded -- Region: Presidente Prudente
data from Adolfo was sucessfully downloaded -- Region: São José do Rio Preto
data from Aguaí was sucessfully downloaded -- Region: Piracicaba
data from Águas da Prata was sucessfully downloaded -- Region: Piracicaba
data from Águas de Lindóia was sucessfully downloaded -- Region: Campinas
data from Águas de Santa Bárbara was sucessfully downloaded -- Region: Sorocaba
data from Águas de São Pedro was sucessfully downloaded -- Region: Piracicaba
data from Agudos was sucessfully downloaded -- Region: Bauru
data from Alambari was sucessfully downloaded -- Region: Sorocaba
data from Alfredo Marcondes was sucessfully downloaded -- Region: Presidente Prudente
data from Altair was sucessfully downloaded -- Region: Ribeirão Preto
data from Altinópolis was sucessfully downloaded -- Region: Ribeirão Preto
data from Alto Alegre was sucessfully downloaded -- Region: Araçatuba
data from Alumínio was sucessfully downloaded

In [4]:
import geocoder
import glob

df_agg = pd.DataFrame(columns=['Jan','Fev','Mar','Abr','Mai','Jun','Jul','Ago','Set','Out','Nov','Dez','Total','Unnamed: 14','City','Pop','Long','Lat'])

current_folder = globals()['_dh'][0]  + '\\csv\\'

for file in (glob.glob(current_folder + "/*.csv")):

    city_name = file
    city_name = city_name.split('-')[1].split('.')[0].split('Município ')[1]

    g = geocoder.geonames(city_name,country=['BR'], state=['São Paulo'],key='<yourOwnKey>')

    df = pd.read_csv(file,sep=';',encoding='UTF-16 LE',skiprows=1)

    df['City'] = city_name
    df['Long'] = g.lng
    df['Lat'] = g.lat
    df['Pop'] = g.population
    df_agg = pd.concat([df_agg,df])

    print('City: {city_name} -------- State: {state_name} ---------- Population: {pop}'.format(city_name=city_name,state_name=g.state,pop=g.population))

    df_agg

df_agg.to_excel('/data/Data.xlsx')

City: Adamantina -------- State: São Paulo ---------- Population: 35111
City: Adolfo -------- State: São Paulo ---------- Population: 3554
City: Aguaí -------- State: São Paulo ---------- Population: 36648
City: Agudos -------- State: São Paulo ---------- Population: 37401
City: Alambari -------- State: São Paulo ---------- Population: 6129
City: Alfredo Marcondes -------- State: São Paulo ---------- Population: 3891
City: Altair -------- State: São Paulo ---------- Population: 4186
City: Altinópolis -------- State: São Paulo ---------- Population: 15609
City: Alto Alegre -------- State: Roraima ---------- Population: 15380
City: Alumínio -------- State: São Paulo ---------- Population: 18767
City: Alvinlândia -------- State: São Paulo ---------- Population: 3237
City: Americana -------- State: São Paulo ---------- Population: 196022
City: Amparo -------- State: São Paulo ---------- Population: 65836
City: Américo Brasiliense -------- State: São Paulo ---------- Population: 34331
City: